In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle, glob
import pp, pf
import cna
import paths
plt.style.use('../pp.mplstyle')

# Utility functions

In [6]:
def read(simname):
    df = pd.DataFrame()

    for n in noises:
        folder = pf.simresults(dset, simname.format(n))
        for m in methods:
            rs = [
                    pickle.load(open(f, 'rb'))
                    for f in glob.glob('{}/{}/*.p'.format(folder, m))
                ]
            rs = pd.DataFrame({
                'id':[r.id for r in rs],
                'p':[r.p for r in rs],
                'interp':[r.interp for r in rs],
                'pheno':[r.pheno for r in rs],
                'noise':[n for r in rs],
                'method':[m for r in rs]
            })
            df = df.append(rs, ignore_index=True)
    print('\tfound', len(df), 'results across', len(df.method.unique()), 'methods,', len(df.noise.unique()), 'noise levels, and',
         len(df.pheno.unique()), 'phenos')
    return df

def aggregate(df):
    # aggregate
    results = dict()
    for n in noises:
        results[n] = df[df.noise == n][['id','method','p','interp']].pivot(index='id', columns='method')

        # add best-of-MASC results
        results[n]['p','MASC_best'] = results[n]['p'][MASCs].min(axis=1, skipna=False)*len(MASCs)
        results[n]['interp','MASC_best'] = results[n]['interp'][MASCs].mean(axis=1, skipna=False)
        results[n] = results[n][np.isfinite(results[n].p.MASC_best)]
    return results

def calculate(results):
    # calculate power and interpretability w std errors
    allmethods = methods+['MASC_best']
    power = pd.DataFrame(index=allmethods, columns=pd.MultiIndex.from_product([['avg', 'se'], noises]))
    for n in noises:
        power['avg', n] = np.nanmean(results[n].p[allmethods] <= 0.05, axis=0)
        power['se', n] = np.nanstd(results[n].p[allmethods] <= 0.05, axis=0) / np.sqrt(results[n].p[allmethods].notna().sum())

    interp = pd.DataFrame(index=allmethods, columns=pd.MultiIndex.from_product([['avg', 'se'], noises]))
    for n in noises:
        interp['avg', n] = results[n].interp[allmethods].mean(axis=0)
        interp['se', n] = results[n].interp[allmethods].std(axis=0) / np.sqrt(len(results[n]))
        
    return power, interp

In [37]:
def umap(z, mask, ax, ix=None):
    if ix is None: ix = np.repeat([True], len(z))
    z = z[ix][mask[ix]]
    ax.scatter(*d.obsm['X_umap'][ix,:2].T, color='gray', alpha=0.05, **pp.umapprops)
    ax.scatter(*d.obsm['X_umap'][ix,:2][mask[ix]].T, c=z, alpha=0.05, cmap='seismic', vmin=-1, vmax=1, **pp.umapprops)
    ax.axis('off')

def powerplot(power, ax, methodinfo, legend=False, ylabels=False, legendloc=4):
    for m,l,c in methodinfo:
        ax.plot(snr, power.loc[m].avg, color=c, label=l, marker='.', ms=1, linewidth = 0.5)
        ax.fill_between(snr, power.loc[m].avg - power.loc[m].se, power.loc[m].avg + power.loc[m].se, 
                        color=c, alpha=0.4, lw=0) 
    ax.set_xlim(0.01, 1) 
    ax.set_xticks([])
    ax.set_ylim(0,1.1)
    if ylabels:
        ax.set_yticks([0,1]); ax.set_yticklabels(['0.0', '1.0'])
        ax.set_ylabel('Power')
    else:
        ax.set_yticks([])
    if legend: ax.legend(loc=legendloc)

def interpplot(interp, ax, methodinfo, legend=False, xlabel=False, xticks=True, ylabels=False, ymin=0, legendloc=4,
              xlabelname='Signal-to-Noise Ratio'):
    for m,l,c in methodinfo:
        ax.plot(snr, interp.loc[m].avg, color=c, label=l, marker='.', ms=1, linewidth = 0.5)
        ax.fill_between(snr, interp.loc[m].avg - interp.loc[m].se, interp.loc[m].avg + interp.loc[m].se, 
                        color=c, alpha=0.4, lw=0)
    ax.set_xlim(0.01, 1)
    ax.set_xticks([])
    if xlabel:
        ax.set_xlabel(xlabelname)
    if xticks:
        ax.set_xticks([0.01,1])
        
    ax.set_ylim(ymin,0.55)
    ax.axhline(y=0, color="gray", linestyle="--", alpha=0.6, linewidth=0.7)
    if ylabels:
        ax.set_yticks([ymin,0,0.5])
        ax.set_ylabel('Signal recovery')
    else:
        ax.set_yticks([])
    if legend: ax.legend(loc=legendloc)
        
def make_xls(power, interp, phenoname, writer):
    numerical = power.rename({
                'avg':'avg power as function of noise/signal ratio',
                'se':'standard error of estimate of average power'}, axis=1
            ).rename({
                'CNAfast':'CNA',
                'MASC_leiden0p2':'MASC (res=0.2)',
                'MASC_leiden1':'MASC (res=1)',
                'MASC_leiden2':'MASC (res=2)',
                'MASC_leiden5':'MASC (res=5)',
                'MASC_best':'MASC (best)'}
            )
    numerical.to_excel(writer, 'power, {}'.format(phenoname))

    numerical = interp.rename({
                'avg':'avg signal recovery as function of noise/signal ratio',
                'se':'standard error of estimate of average signal recovery'}, axis=1
            ).rename({
                'CNAfast':'CNA',
                'MASC_leiden0p2':'MASC (res=0.2)',
                'MASC_leiden1':'MASC (res=1)',
                'MASC_leiden2':'MASC (res=2)',
                'MASC_leiden5':'MASC (res=5)',
                'MASC_best':'MASC (best)'}
            )
    numerical.to_excel(writer, 'sig rec, {}'.format(phenoname))

# TBRU

## set parameters

In [8]:
d = cna.read('/data/srlab1/laurie-yakir/tbru.real/2.munged/harmcca20.h5ad')

dset = 'harmcca20'
noises = [1,
    1.11,
    1.25,
    1.43,
    1.67,
    2,
    2.5,
    3.33,
    5,
    10,
    99,
]
snr = 1/np.array(noises)

methods = [
    'CNAfast',
    'MASC_leiden0p2',
    'MASC_leiden1',
    'MASC_leiden2',
    'MASC_leiden5',
]
MASCs = methods[1:]

## main power/interpretability figure

In [9]:
# choose example phenotypes to visualize in figure
clust = 5
mask = d.obs.leiden2==str(clust)

# causalclust
z_causalclust = np.zeros(len(d))
z_causalclust[mask] = 1

# GEpc
z_geneexpPCs = d.X[:,0]

# Make cluster_specific PC1 example
import scanpy as sc
z_clusterspecificPC1 = np.zeros(len(d))
z_clusterspecificPC1[mask]=sc.pp.pca(data=d.X[mask,:], n_comps=2)[:,0]

In [36]:
writer = pd.ExcelWriter('../_xls/sim.xlsx')
fig, axs = plt.subplots(3,3, figsize=(6,4))
methodinfo = [('CNAfast', 'CNA', 'blue'), ('MASC_best', 'Clustering', 'green')]

for i, (simname, z, mask, phenoname) in enumerate([
    ('clusterabundance/noise_{}/causalclust_leiden2', z_causalclust, mask, 'cluster'),
    ('geneexpPCs/noise_{}', z_geneexpPCs, np.ones(len(d)).astype(bool), 'glob GE prog'),
    ('clusterspecificPC1/noise_{}/causalclust_leiden1', z_clusterspecificPC1, mask, 'clust-spec GE prog'),
    ]):
    print(simname)
    umap(z, mask, axs[0,i])
    df = read(simname)
    results = aggregate(df)
    power, interp = calculate(results)
    make_xls(power, interp, phenoname, writer)
    
    # Report selected values
    snr_sel = np.where(snr==0.5)[0] # Signal:Noise ratio of 50%
    print(power.iloc[:,snr_sel])
    print(interp.iloc[:,snr_sel])
    
    powerplot(power, axs[1,i], methodinfo,
            ylabels=i==0, legend=i==0)
    interpplot(interp, axs[2, i], methodinfo, ymin=-0.1,
            xlabel=i==1, ylabels=i==0, legend=i==0)

ax = axs[0,0]
ax.set_ylabel("Simulated Values")
ax.set_title("Causal Cluster")
ax = axs[0,1]
ax.set_title("Global\nExpression Program")
ax = axs[0,2]
ax.set_title("Cluster-Specific\nExpression Program")
plt.tight_layout()
plt.savefig('../_figs/rawmainfig.sim.power_interp.pdf')
writer.save()

clusterabundance/noise_{}/causalclust_leiden2
	found 13200 results across 5 methods, 11 noise levels, and 24 phenos
                     avg
                     2.0
CNAfast         1.000000
MASC_leiden0p2  0.816667
MASC_leiden1    0.995833
MASC_leiden2    1.000000
MASC_leiden5    1.000000
MASC_best       1.000000
                     avg
                     2.0
CNAfast         0.424171
MASC_leiden0p2  0.196051
MASC_leiden1    0.423395
MASC_leiden2    0.495560
MASC_leiden5    0.442337
MASC_best       0.389336
geneexpPCs/noise_{}
	found 11000 results across 5 methods, 11 noise levels, and 20 phenos
                  avg
                  2.0
CNAfast         0.995
MASC_leiden0p2  0.400
MASC_leiden1    0.835
MASC_leiden2    0.795
MASC_leiden5    0.780
MASC_best       0.775
                     avg
                     2.0
CNAfast         0.447839
MASC_leiden0p2  0.119707
MASC_leiden1    0.341905
MASC_leiden2    0.365235
MASC_leiden5    0.381651
MASC_best       0.302125
clusterspecificPC1

## supplementary power/interpretability figure

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(6,2/3*3.75))
methodinfo = [('CNAfast', 'CNA', 'blue'), 
              ('MASC_leiden0p2', 'Leiden 0.2', 'gold'), ('MASC_leiden1', 'Leiden 1', 'fuchsia'),
              ('MASC_leiden2', 'Leiden 2', 'red'), ('MASC_leiden5', 'Leiden 5', 'purple')]

for i, simname in enumerate([
    'clusterabundance/noise_{}/causalclust_leiden2',
    'geneexpPCs/noise_{}',
    'clusterspecificPC1/noise_{}/causalclust_leiden1',
    ]):
    print(simname)
    df = read(simname)
    results = aggregate(df)
    power, interp = calculate(results)
    powerplot(power, axs[0,i], methodinfo,
                    ylabels=i==0, legend=i==0)
    interpplot(interp, axs[1, i], methodinfo,
                    xlabel=i==1, ylabels=i==0, legend=i==0, ymin=-0.1)
axs[0,0].set_title('Causal Cluster')
axs[0,1].set_title('Global\nExpression Program')
axs[0,2].set_title('Cluster-Specific\nExpression Program')
plt.tight_layout()
plt.savefig('../_figs/suppfig.sim.power_interp.pdf', dpi=1000)

## supplementary illustration of interpretability

In [ ]:
# Define true scores
n_clust = 0
true_scores = np.repeat(0,d.obs.shape[0])
true_scores[d.obs['aparna2p0']==n_clust] = 1
d.obs['true_scores'] = true_scores
d.obs_to_sample(['true_scores'])

# High interpretability estimate (low noise)
batches = d.samplem.batch.values
res = cna.tl._association.association(d, d.samplem['true_scores'], batches=batches, covs=None, Nnull=1000,
                                     local_test = True, force_recompute=True)
d.obs['highinterp'] = np.repeat(0,d.obs.shape[0])
d.obs['highinterp'].loc[d.uns['keptcells']] = res.ncorrs
d.obs['highinterp'].loc[d.obs['highinterp']<res.fdr_5p_t] = 0

In [ ]:
# Low interpretability estimate (different cluster+noise)

# Define decoy scores
n_clust = 1
true_scores = np.repeat(0,d.obs.shape[0])
true_scores[d.obs['aparna2p0']==n_clust] = 1
d.obs['decoy_scores'] = true_scores
d.obs_to_sample(['decoy_scores'])

# Add noise
noiselevels = 4
Yvar = np.std(d.obs['decoy_scores'])
noiselevels = noiselevels * Yvar
noise = np.random.randn(d.obs['decoy_scores'].shape[0]) * noiselevels
d.obs['noisy_scores'] = d.obs['decoy_scores']+noise
d.obs_to_sample(['noisy_scores'])

# Low interpretability estimate
res = cna.tl._association.association(d, d.samplem['noisy_scores'], batches=batches, covs=None, Nnull=1000,
                                     local_test = True)
d.obs['lowinterp'] = np.repeat(0,d.obs.shape[0])
d.obs['lowinterp'].loc[d.uns['keptcells']] = res.ncorrs
d.obs['lowinterp'].loc[d.obs['lowinterp']<res.fdr_5p_t] = 0

In [ ]:
# Correlation of high interpretability estimate
high_int_R = np.around(np.corrcoef(d.obs['true_scores'], d.obs['highinterp'])[0,1],2)

In [ ]:
# Correlation of low interpretability estimate
low_int_R = np.around(np.corrcoef(d.obs['true_scores'], d.obs['lowinterp'])[0,1],2)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(3,1))

#ix = np.random.choice(np.arange(d.obs.shape[0]), 10000, replace = False)
mask = (d.obs['true_scores']!=0).values
umap(d.obs['true_scores'].values, mask, axs[0])
mask = (d.obs['highinterp']!=0).values
umap(d.obs['highinterp'].values, mask, axs[1])
mask = (d.obs['lowinterp']!=0).values
umap(d.obs['lowinterp'], mask, axs[2])
ax = axs[0]
ax.set_title('Simulated\nTrue Values', fontsize = 5)
ax = axs[1]
ax.set_title('High Accuracy\nEstimate', fontsize = 5)
ax.text(0.7, 0.89, '$R = {:.2f}$'.format(high_int_R),
        transform=ax.transAxes, fontsize=4, color="black")
ax = axs[2]
ax.set_title('Low Accuracy\nEstimate', fontsize = 5)
ax.text(0.7, 0.89, '$R = {:.2f}$'.format(low_int_R),
        transform=ax.transAxes, fontsize=4, color="black")


plt.tight_layout()
plt.savefig('../_figs/suppfig.sim.defn_interp.pdf')

## supplementary power/interpretability with downsampling

In [ ]:
dsets = [
    'harmcca20',
    'harmcca20_B18s0',
    'harmcca20_B12s0',
    'harmcca20_B8s0',
]
dsetnames = [
    'TBRU full, N='+str(cna.read(paths.simdata + dsets[0] + '.h5ad').N),
    'TBRU, N='+str(cna.read(paths.simdata + dsets[1] + '.h5ad').N),
    'TBRU, N='+str(cna.read(paths.simdata + dsets[2] + '.h5ad').N),
    'TBRU, N='+str(cna.read(paths.simdata + dsets[3] + '.h5ad').N),
]

noises = [1,
    1.11,
    1.25,
    1.43,
    1.67,
    2,
    2.5,
    3.33,
    5,
    10,
    99,
]
snr = 1/np.array(noises)

methods = [
    'CNAfast',
    'MASC_leiden0p2',
    'MASC_leiden1',
    'MASC_leiden2',
    'MASC_leiden5',
]
MASCs = methods[1:]

In [ ]:
fig, axs = plt.subplots(8,3, figsize=(6,4*2/3*3.75))
for row, (dset, dsetname) in enumerate(zip(dsets, dsetnames)):
    methodinfo = [('CNAfast', 'CNA', 'blue'), ('MASC_best', 'Clustering', 'green')]
    for i, simname in enumerate([
        'clusterabundance/noise_{}/causalclust_leiden2',
        'geneexpPCs/noise_{}',
        'clusterspecificPC1/noise_{}/causalclust_leiden1',
        ]):
        print(dset, simname)
        df = read(simname)
        results = aggregate(df)
        power, interp = calculate(results)
        powerplot(power, axs[2*row,i], methodinfo,
                        ylabels=i==0, legend=i==0, legendloc=2)
        interpplot(interp, axs[2*row+1, i], methodinfo,
                        xlabel=((i==1) & (row==3)), xticks=row==3, ylabels=i==0, legend=i==0, legendloc=2, ymin=-0.1,
                        xlabelname='Signal-to-noise ratio')
    axs[2*row,1].set_title(dsetname)
axs[0,0].set_title('Causal Cluster\n\n'+axs[0,0].get_title())
axs[0,1].set_title('Global\nExpression Program\n\n'+axs[0,1].get_title())
axs[0,2].set_title('Cluster-Specific\nExpression Program\n\n'+axs[0,2].get_title())
plt.tight_layout()
plt.savefig('../_figs/suppfig.sim.power_interp_tbrudownsampled.pdf', dpi=1000)

# Sepsis

## set parameters

In [2]:
d = cna.read('/data/srlab1/laurie-yakir/sim/datasets/sepsis_pbmc.h5ad')

dsets = [
    'sepsis_pbmc',
    'sepsis_pbmc_N40s0',
    'sepsis_pbmc_N20s0',
]
dsetnames = [
    'sepsis full, N='+str(d.N),
    'sepsis, N=40',
    'sepsis, N=20'
]

noises = [
    0.001,
    0.111,
    0.25,
    0.43,
    0.67,
    1,
    1.5,
    2.33,
    4,
    9,  
    99,
]
snr = 1/(1+np.array(noises))

methods = [
    'CNAfast',
    'MASC_leiden0p2',
    'MASC_leiden1',
    'MASC_leiden2',
    'MASC_leiden5',
]

MASCs = methods[1:]

## supplementary power/interpretability figure

In [ ]:
fig, axs = plt.subplots(6,3, figsize=(6,3*2/3*3.75))
for row, (dset, dsetname) in enumerate(zip(dsets, dsetnames)):
    methodinfo = [('CNAfast', 'CNA', 'blue'), ('MASC_best', 'Clustering', 'green')]
    for i, simname in enumerate([
        'clusterabundance/noise_{}/causalclust_leiden2',
        'geneexpPCs/noise_{}',
        'clusterspecificPC1/noise_{}/causalclust_leiden1',
        ]):
        print(dset, simname)
        df = read(simname)
        results = aggregate(df)
        power, interp = calculate(results)
        powerplot(power, axs[2*row,i], methodinfo,
                        ylabels=i==0, legend=i==0, legendloc=2)
        interpplot(interp, axs[2*row+1, i], methodinfo,
                        xlabel=((i==1) & (row==2)), xticks=row==2, ylabels=i==0, legend=i==0, legendloc=2, ymin=-0.1,
                        xlabelname='Signal strength (% variance explained)')
    axs[2*row,1].set_title(dsetname)
axs[0,0].set_title('Causal Cluster\n\n'+axs[0,0].get_title())
axs[0,1].set_title('Global\nExpression Program\n\n'+axs[0,1].get_title())
axs[0,2].set_title('Cluster-Specific\nExpression Program\n\n'+axs[0,2].get_title())
plt.tight_layout()
plt.savefig('../_figs/suppfig.sim.power_interp_sepsisdownsampled.pdf', dpi=1000)

In [ ]:
#optional
#check how many simulations finished for each dataset/signal type
sims = [
    'clusterabundance/noise_{}/causalclust_leiden2',
    'geneexpPCs/noise_{}',
    'clusterspecificPC1/noise_{}/causalclust_leiden1'
]

fig, axs = plt.subplots(3,3, figsize=(9,9), dpi=100)
axs = axs.flatten()
i = 0
for d in dsets:
    for s in sims:
        dset = d
        df = read(s)
        ax = axs[i]
        for n in noises:
            nums = df[df.noise==n].method.value_counts()
            for m in methods:
                if m not in nums: nums[m] = 0
            ax.plot(nums.sort_index(), label=n)
        ax.set_ylim(0)
        ax.legend()
        ax.set_title(dset + ' ' + s, fontsize=5)
        i += 1
plt.tight_layout()
plt.show()